In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import os
from mysql.connector import Error
from datetime import datetime, date, timedelta
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Definimos la ruta en donde queremos guardar los archivos
ruta_guardar = '/home/estadistico/Documents/Erick/Reportes diarios'
ruta_asig_baz = '/home/estadistico/Documents/Erick/Banco Azteca/Asignacion csv'
ruta_pagos = '/home/estadistico/Documents/Erick'
mes = 20201201
mes2 = 'Diciembre2020'
mes3 = pd.to_datetime('01/12/2020',dayfirst=True)
prim_sem_mes = 49

In [3]:
#Agregamos las variables a ocupar
servidor = '192.168.15.12'
puerto = int('3306')
usuario = 'estadisticas'
contraseña = 'estadisticas8474'
base = 'procesos_externos'

In [4]:
#Lectura pagos Bradesco
pagos_brad = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Bradesco/Pagos/ENRIQUEZ Acumulado de pagos Diciembre de 2020.xlsx')
pagos_brad = pagos_brad.iloc[:,0:9]
pagos_brad = pagos_brad.drop_duplicates()
pagos_brad2 = pd.pivot_table(pagos_brad,index=['NUMERO_TARJETA','FECHA_PAGO','TIPO_ASIGANCION'],values=['MONTO'],aggfunc=np.sum)
pagos_brad2 = pd.DataFrame(pagos_brad2.to_records())
pagos_brad2 = pagos_brad2.loc[pagos_brad2['FECHA_PAGO']>=mes3]

In [5]:
#Lectura pagos Liverpool
sheet_liv = ['SA002REM','SA001REM','DKN04REM','DKN02REM','DKN00REM']
pagos_liv = pd.DataFrame()
for i in sheet_liv:
    pagos_liv_new = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Liverpool/Pagos/PAGOS DIC LIVERPOOL .xlsx',sheet_name = str(i))
    pagos_liv = pd.concat([pagos_liv,pagos_liv_new],ignore_index=True)
pagos_liv = pagos_liv.drop_duplicates()
pagos_liv = pagos_liv.iloc[:,0:6]
pagos_liv.columns = ['Id','Cuenta','Fecha Prom','Fecha Pago','Monto','Posteo']
try: 
    pagos_liv['Fecha Pago'] = pd.to_datetime(pagos_liv['Fecha Pago'],dayfirst=True)
except: pagos_liv['Fecha Pago']   
pagos_liv['Fecha Pago'] = [datetime.strftime(x,'%d/%m/%Y') for x in pagos_liv['Fecha Pago']]
try: 
    pagos_liv['Fecha Prom'] = pd.to_datetime(pagos_liv['Fecha Prom'],dayfirst=True)
except: pagos_liv['Fecha Prom']    
pagos_liv['Fecha Prom'] = [datetime.strftime(x,'%d/%m/%Y') for x in pagos_liv['Fecha Prom']]
pagos_liv2 = pd.pivot_table(pagos_liv,index=['Id','Cuenta','Fecha Pago','Fecha Prom'],values=['Monto'],aggfunc=np.sum)
pagos_liv2 = pd.DataFrame(pagos_liv2.to_records())
#pagos_liv2 = pagos_liv2.loc[pagos_liv2['Fecha Pago']>=mes3]

In [6]:
#Lectura pagos Credifiel
sheet_cred = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Pagos/')
pagos_cred = pd.DataFrame()
for j in sheet_cred:
    pagos_cred_new = pd.read_excel(''+ruta_pagos+'/'+str(mes2)+'/Credifiel/Pagos/'+j+'')
    pagos_cred = pd.concat([pagos_cred,pagos_cred_new],ignore_index=True)
pagos_cred = pagos_cred.iloc[:,[12,1,8]]
pagos_cred2 = pd.pivot_table(pagos_cred,index=['ID','FECHA'],values=['IMPORTE'],aggfunc=np.sum)
pagos_cred2 = pd.DataFrame(pagos_cred2.to_records())

In [7]:
#Lectura pagos Banco Azteca
sheet_baz = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/')
pagos_baz = pd.DataFrame()
for k in sheet_baz:
    pagos_baz_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Pagos/'+k+'')
    pagos_baz = pd.concat([pagos_baz,pagos_baz_new],ignore_index=True)
pagos_baz = pagos_baz.drop_duplicates()
pagos_baz = pagos_baz.loc[pagos_baz['FNMONTOPAGADO']>0]
pagos_baz['FDFECHAABONO'] = pd.to_datetime(pagos_baz['FDFECHAABONO'],dayfirst=True)
pagos_baz = pagos_baz.loc[pagos_baz['FDFECHAABONO']>=mes3]
pagos_baz2 = pd.pivot_table(pagos_baz,index=['CAMPANAID','FDFECHAABONO','FIIDPERIODO','FNSCOMPROMISO','FCEMPNUMCORTE','FNMONTOPROMETIDO'],values=['FNMONTOPAGADO'],aggfunc=np.sum)
pagos_baz2 = pd.DataFrame(pagos_baz2.to_records())

In [8]:
#Leemos facturacion Banco Azteca
sheet_baz2 = os.listdir(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Facturacion/')
fact_baz = pd.DataFrame()
for k in sheet_baz2:
    fact_baz_new = pd.read_csv(''+ruta_pagos+'/'+str(mes2)+'/Banco Azteca/Facturacion/'+k+'',encoding='latin-1')
    fact_baz = pd.concat([fact_baz,fact_baz_new],ignore_index=True)
fact_baz = fact_baz.drop_duplicates()
fact_baz = fact_baz.loc[fact_baz['Número Semana']>=prim_sem_mes]
fact_baz['Final de Semana'] = pd.to_datetime(fact_baz['Final de Semana'],dayfirst=True)
fact_baz['Inicio de Semana'] = pd.to_datetime(fact_baz['Inicio de Semana'],dayfirst=True)

In [9]:
#Asignamos valores a los parametros \n"
today = date.today().strftime('%Y%m%d')
now = datetime.now().strftime('%d-%m-%Y %H-%M')
hoy = np.datetime64('today')
ayer = np.datetime64(date.today() - timedelta(1))
today2 = date.today().strftime('%d-%m-%Y')

In [60]:
#Hacemos la conexion con el servidor\n",
try:
    conn = mysql.connector.connect(user=usuario,
                               password=contraseña,
                               host=servidor,
                               port=3306,
                               database=base)
    conn.set_charset_collation('latin1')
except mysql.Error as e:
    print("Failed to execute stored procedure: {}".format(error))

cursor = conn.cursor()

In [11]:
#Hacemos la asignacion de cada una de las metas para cambiar de arancel sin tomar el 0 en cuenta
c04pv1, arancel_4pv1 = 1204704, .07
c04pv2, arancel_4pv2 = 1807056, .09
c04pv3, arancel_4pv3 = 2409407, .12
c05pv1, arancel_5pv1 = 662269, .09
c05pv2, arancel_5pv2 = 993403, .10
c05pv3, arancel_5pv3 = 1324538, .14
c06pv1, arancel_6pv1 = 807172, .10
c06pv2, arancel_6pv2 = 1210759, .11
c06pv3, arancel_6pv3 = 1614345, .16
a1, arancel_a1 = 260190, .27
a2, arancel_a2 = 390285, .30
a3, arancel_a3 = 520381, .33
b1, arancel_b1 = 215083, .25
b2, arancel_b2 = 322624, .28
b3, arancel_b3 = 430165, .31
c11, arancel_c11 = 1430989, .17
c12, arancel_c12 = 2146483, .20
c13, arancel_c13 = 2861978, .23
c21, arancel_c21 = 626634, .19
c22, arancel_c22 = 939951, .22
c23, arancel_c23 = 1253267, .25
c31, arancel_c31 = 246425, .21
c32, arancel_c32 = 369637, .24
c33, arancel_c33 = 492849, .27
c41, arancel_c41 = 123529, .23
c42, arancel_c42 = 185294, .26
c43, arancel_c43 = 247059, .29

In [12]:
#Funcion para hacer la consulta sql, ya sea de gestion o de asignacion en donde requerimos tres parametros, la tabla de donde vamos a sacar la consulta, en caso de asignacin requerimos los otros dos parametros fecha inicial y fecha final
def consulta_sql(table,from_date=None,to_date=None):
    if from_date is None:
        sql_mes = cursor.callproc(table,)
        for result in cursor.stored_results():
            sql_mes = pd.DataFrame(result.fetchall())
    else:
        sql_mes = cursor.callproc(table, [from_date,to_date])
        for result in cursor.stored_results():
            sql_mes = pd.DataFrame(result.fetchall())
    sql_mes = sql_mes.drop_duplicates()
    return sql_mes

In [13]:
#Funcion para transformar varias columnas a numero
def transform_numeric(df,cols):
    return df[cols].apply(pd.to_numeric, errors='coerce')

#Funcion que convierte a dataframe y rellena espacios con 0
def to_dataframe(df):
    df = pd.DataFrame(df.to_records())
    df.fillna(0,inplace=True)
    return df

# Reporte Liverpool

In [14]:
#Hacemos la consulta referente a las gestiones de Liverpool
gestion_liv_mes = consulta_sql('liverpool_rpt_gestiones_detallado',mes,today)
gestion_liv_mes.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','fecha_promesa','monto_promesa','comentarios']
gestion_liv_mes = gestion_liv_mes.loc[gestion_liv_mes['dictamen']=='PROMESA']
gestion_liv_mes[['credito','telefono']] = transform_numeric(gestion_liv_mes,['credito','telefono'])
gestion_liv_mes['anio'] = [int(x[:4]) for x in gestion_liv_mes['fecha_promesa']]
gestion_liv_mes = gestion_liv_mes.loc[(gestion_liv_mes['anio']>2010)&(gestion_liv_mes['anio']<2100)]
gestion_liv_mes['fecha_promesa'] = pd.to_datetime(gestion_liv_mes['fecha_promesa'],dayfirst=False)

In [15]:
gestion_liv_mes[['credito','telefono']] = transform_numeric(gestion_liv_mes,['credito','telefono'])

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion,resultado,fecha_promesa,monto_promesa,comentarios,anio
1,9371500,12,5,13000008692803,JOSE JORGE ANTONIO FLORES FRANCO,5554109802,celular,2020-12-14 17:14:54,BARRETOOSMF,BARRETO OSEGUERA MARIA FERNANDA,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,2020-12-15,9500.00,"5554109802 // PROMESA DE PAGO LIQUIDACION $9,...",2020
15,9371399,12,4,13000028312853,MAYRA ACEVEDO CARDENAS,4481058612,celular,2020-12-14 17:16:01,LOCISNEROSLE,LUCIA ESMERALDA LONGORIA CISNEROS,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,2020-12-15,3634.00,4481058612 TT SE COMUNICA POR WHATS POR QUE L...,2020
16,9371396,12,4,4178490015195523,MAYRA ACEVEDO CARDENAS,4481058612,celular,2020-12-14 17:08:09,LOCISNEROSLE,LUCIA ESMERALDA LONGORIA CISNEROS,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,2020-12-15,5155.00,4481058612 TT SE COMUNICA POR QUE LE LLEGA UN ...,2020
100,9370925,12,4,13000004259748,JORGE EDUARDO CUELLAR MARQUEZ,8448691707,celular,2020-12-14 16:41:50,BARRETOOSMF,BARRETO OSEGUERA MARIA FERNANDA,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,2020-12-30,6500.00,"8448691707 // PROMESA LIQUIDACION $6,500 30/1...",2020
166,9370620,12,4,4178490013749404,ELIZABETH LOPEZ RODRIGUEZ,5621852906,celular,2020-12-14 16:18:00,GARCIARUAL,GARCIA RUIZ ALEXIS,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-18,400.00,5621852906 tt estara realizando pagos semanles...,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23893,9234152,12,5,53000000271987,ALAN JAVIER LOPEZ ORTIZ,6564108412,celular,2020-12-01 08:05:04,SANCHEZVEVAIV,SANCHEZ VELASCO VANESSA IVONNE,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,2020-12-01,6000.00,"6564108412 TT PL POR $6,000 SE OFRECE LIQUID...",2020
23955,9232798,12,1,4178490007802342,FELIX MOGUEL SOSA,9711040827,celular,2020-12-01 10:24:57,TONARVAEZMA,TORRES NARVAEZ MAYTE,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-12-15,3750.00,9711040827 TT REALIZARA SIG PAGO EL DIA 15/1...,2020
23999,9231922,12,5,4178490002481845,PAOLA MORENO ESTRADA,9611121706,celular,2020-12-01 09:38:11,OSNAYAVALOGU,LORAYNI GUADALUPE OSNAYA VAZQUEZ,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,2020-12-01,4000.00,9611121706PP LIQYUIDACION POR LA CANTIDAD DE $...,2020
24050,9231349,12,4,13000031192839,ENRIQUE RAMIREZ CHAVEZ,4621913856,celular,2020-12-01 09:16:18,OSNAYAVALOGU,LORAYNI GUADALUPE OSNAYA VAZQUEZ,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,2020-12-15,4234.00,4621913856 PP LIQUIDACION POR LA CANTIDAD DE $...,2020


In [16]:
#Hacemos la consulta de la asignacion
sql_asig_liv = cursor.callproc('liverpool_rpt_asignacion_activa',)
for result in cursor.stored_results():
    asignacion_liv = pd.DataFrame(result.fetchall())
asignacion_liv = asignacion_liv.iloc[:,[0,1,3,24,6,26]]
asignacion_liv.columns = ['firma_id','unegocio_id','credito','estado','rfc','division']
asignacion_liv['credito'] = pd.to_numeric(asignacion_liv['credito'])

In [17]:
#Hacemos el consolidado
consolidado_liv_mes = pd.merge(gestion_liv_mes,asignacion_liv,how='left',on=['firma_id','unegocio_id','credito'])
cols = ['monto_promesa','credito']
consolidado_liv_mes[cols] = consolidado_liv_mes[cols].apply(pd.to_numeric, errors='coerce')
consolidado_liv_mes = consolidado_liv_mes[(consolidado_liv_mes['monto_promesa']>50)]
consolidado_liv_mes = consolidado_liv_mes[(consolidado_liv_mes['monto_promesa']<500000)]
consolidado_liv_mes['division'].fillna('COBRANZA',inplace=True)
consolidado_liv_mes['division'] = consolidado_liv_mes['division'].replace('','COBRANZA')
consolidado_liv_mes = consolidado_liv_mes.drop_duplicates(subset = ['usuario','credito','fecha_promesa','monto_promesa'])

In [18]:
productos = [('DKN00REM','pren'),('DKN02REM','REM2'),('DKN04REM','REM4'),('SA001REM','SB02'),('SA002REM','REM9')]
productos = pd.DataFrame(productos)
productos.columns = ['id','division']
consolidado_liv_mes = pd.merge(consolidado_liv_mes,productos,how='left',on=['division'])

In [19]:
#Asignamos los aranceles de liverpool
aranceles = [('DKN00REM',0.28),('DKN02REM',0.4),('DKN04REM',0.5),('SA001REM',0.15),('SA002REM',0.16)]
aranceles = pd.DataFrame(aranceles)
aranceles.columns = ['Id','Arancel']

#Agregamos aranceles al consolidado
pagos_liv2 = pd.merge(pagos_liv2,aranceles,how='left',on='Id')
pagos_liv2['PagoArancel'] = [x*y for x,y in zip(pagos_liv2['Monto'],pagos_liv2['Arancel'])]

In [20]:
#Unimos el consolidado con los pagos 
consolidado_pagos_liv = pd.merge(consolidado_liv_mes,pagos_liv2,how='left',left_on=['credito'],right_on=['Cuenta'])
consolidado_pagos_liv = consolidado_pagos_liv.drop_duplicates(subset = ['credito','Monto','usuario','fecha_promesa'])
consolidado_pagos_liv['Fecha Pago'] = pd.to_datetime(consolidado_pagos_liv['Fecha Pago'],dayfirst=True)
consolidado_pagos_liv['dia_prom'] = consolidado_pagos_liv['fecha_promesa'].dt.day
consolidado_pagos_liv['dia_pago'] = consolidado_pagos_liv['Fecha Pago'].dt.day
cols = ['dia_prom','dia_pago','Monto']
consolidado_pagos_liv[cols] = consolidado_pagos_liv[cols].apply(pd.to_numeric, errors='coerce',downcast='integer')
consolidado_pagos_liv['dia_pago'].fillna(0,inplace=True)
try:
    consolidado_pagos_liv['dia_pago'] = [int(x) for x in consolidado_pagos_liv['dia_pago']]
except: consolidado_pagos_liv['dia_pago'] = consolidado_pagos_liv['dia_pago']

In [21]:
#Condicionamos a los gestores cuyas promesas y pagos estan en un plazo de 3 dias
consolidado_pagos_liv['PagoReal'] = [np.where((z < x+pd.Timedelta(3, unit='D')) & ((z > x-pd.Timedelta(3, unit='D') )), y, 0) for x,y,z in zip(consolidado_pagos_liv['fecha_promesa'],consolidado_pagos_liv['Monto'],consolidado_pagos_liv['Fecha Pago'])]
consolidado_pagos_liv['PagoReal'] = [x.item() for x in consolidado_pagos_liv['PagoReal']]

In [22]:
consolidado_pagos_liv['MontoArancel'] = [x*y for x,y in zip(consolidado_pagos_liv['PagoReal'],consolidado_pagos_liv['Arancel'])]

In [23]:
#Obtenemos datos de los gestores de Liverpool
promesas_mes_liv = pd.pivot_table(consolidado_pagos_liv,index=['id'],values=['monto_promesa'],aggfunc=['count',np.sum],margins=True,dropna=False)
promesas_mes_liv = pd.DataFrame(promesas_mes_liv.to_records())
promesas_mes_liv.columns = ['Division','NumeroPromesas','SumaPromesas']
promesas_mes_liv.fillna(0,inplace=True)
promesas_mes_liv

,Division,NumeroPromesas,SumaPromesas
0,DKN00REM,104,405864.91
1,DKN02REM,101,521353.88
2,DKN04REM,66,330692.95
3,SA001REM,178,326907.43
4,SA002REM,31,52880.76
5,All,494,1677143.93


In [24]:
#Obtenemos las promesas restantes a partir del día de hoy
restantes_liv = consolidado_pagos_liv.loc[consolidado_pagos_liv['fecha_promesa']>=ayer]
promesas_restantes_liv = pd.pivot_table(restantes_liv,index=['id'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
promesas_restantes_liv = pd.DataFrame(promesas_restantes_liv.to_records())
promesas_restantes_liv.columns = ['Division','PromesasRestantes']
promesas_mes_liv.fillna(0,inplace=True)
promesas_restantes_liv

,Division,PromesasRestantes
0,DKN00REM,202160.00
1,DKN02REM,194309.00
2,DKN04REM,101574.00
3,SA001REM,140887.92
4,SA002REM,14796.00


In [25]:
#Total pagos Liver registrados
pagos_totales_liv = pd.pivot_table(pagos_liv2,index=['Id'],values=['Monto','PagoArancel'],aggfunc=[np.sum,'count'],dropna=False)
pagos_totales_liv = pd.DataFrame(pagos_totales_liv.to_records())
pagos_totales_liv = pagos_totales_liv.iloc[:,[0,3,1,2]]
pagos_totales_liv.columns = ['Division','NumeroPagos','SumaPagos','SumaFacturacion']
pagos_totales_liv.fillna(0,inplace=True)
pagos_totales_liv['FechaFact'] = max(pagos_liv2['Fecha Pago'])
pagos_totales_liv['FechaFact'] = pd.to_datetime(pagos_totales_liv['FechaFact'],dayfirst=True)
pagos_totales_liv

,Division,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact
0,DKN00REM,58,213155.91,59683.6548,2020-12-13
1,DKN02REM,83,380770.64,152308.2560,2020-12-13
2,DKN04REM,36,199262.95,99631.4750,2020-12-13
3,SA001REM,114,197833.62,29675.0430,2020-12-13
4,SA002REM,16,27809.26,4449.4816,2020-12-13


In [26]:
final_liv = pd.merge(pagos_totales_liv,promesas_mes_liv,how='left',on=['Division'])
final_liv = pd.merge(final_liv,promesas_restantes_liv,how='left',on=['Division'])
final_liv.insert(loc=0,column='Cliente',value='Liverpool')
final_liv

,Cliente,Division,NumeroPagos,SumaPagos,SumaFacturacion,FechaFact,NumeroPromesas,SumaPromesas,PromesasRestantes
0,Liverpool,DKN00REM,58,213155.91,59683.6548,2020-12-13,104,405864.91,202160.00
1,Liverpool,DKN02REM,83,380770.64,152308.2560,2020-12-13,101,521353.88,194309.00
2,Liverpool,DKN04REM,36,199262.95,99631.4750,2020-12-13,66,330692.95,101574.00
3,Liverpool,SA001REM,114,197833.62,29675.0430,2020-12-13,178,326907.43,140887.92
4,Liverpool,SA002REM,16,27809.26,4449.4816,2020-12-13,31,52880.76,14796.00


# Reporte Bradesco

In [27]:
#Hacemos la consulta de la asignacion
sql_asig_brad = cursor.callproc('bradescard_rpt_asignacion_activa',)
for result in cursor.stored_results():
    asignacion_brad = pd.DataFrame(result.fetchall())
asignacion_brad = asignacion_brad.iloc[:,[0,1,3,9,24,6,26]]
asignacion_brad.columns = ['firma_id','unegocio_id','credito','saldo_total','estado','rfc','division']
asignacion_brad['credito'] = pd.to_numeric(asignacion_brad['credito'])

In [61]:
#Hacemos la consulta referente a las gestiones de Bradesco
sql_gest_brad_mes = cursor.callproc('bradescard_rpt_gestiones_detallado', [mes,today])
for result in cursor.stored_results():
    gestion_brad_mes = pd.DataFrame(result.fetchall())
gestion_brad_mes.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_brad_mes = gestion_brad_mes.loc[gestion_brad_mes['dictamen']=='PROMESA']
gestion_brad_mes['credito'] = pd.to_numeric(gestion_brad_mes['credito'])
gestion_brad_mes['monto_promesa'] = pd.to_numeric(gestion_brad_mes['monto_promesa'])
gestion_brad_mes = gestion_brad_mes.drop_duplicates(subset=['credito','usuario','fecha_promesa','monto_promesa'])

KeyboardInterrupt: 

In [ ]:
#Hacemos la union de asignacion y promesas
consolidado_brad_mes = pd.merge(gestion_brad_mes,asignacion_brad,how='left',on=['firma_id','unegocio_id','credito'])
cols = ['monto_promesa','credito']
consolidado_brad_mes[cols] = consolidado_brad_mes[cols].apply(pd.to_numeric, errors='coerce')
consolidado_brad_mes = consolidado_brad_mes[(consolidado_brad_mes['monto_promesa']>50)]
consolidado_brad_mes = consolidado_brad_mes[(consolidado_brad_mes['monto_promesa']<500000)]
consolidado_brad_mes.fillna('COBRANZA',inplace=True)

In [ ]:
#Obtenemos los pagos totales por division
pagos_totales_brad = pd.pivot_table(pagos_brad2,index=['TIPO_ASIGANCION'],values=['MONTO'],aggfunc=[np.sum,'count'],margins=True,dropna=False)
pagos_totales_brad = pd.DataFrame(pagos_totales_brad.to_records())
pagos_totales_brad = pagos_totales_brad.iloc[:,[0,2,1]]
pagos_totales_brad.columns = ['Division','NumeroPagos','SumaPagos']
pagos_totales_brad.fillna(0,inplace=True)

In [ ]:
C04PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 4PV', 'SumaPagos'].item()
C05PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 5PV', 'SumaPagos'].item()
C06PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 6PV', 'SumaPagos'].item()
A = pagos_totales_brad.loc[pagos_totales_brad['Division']=='A', 'SumaPagos'].item()
C1 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C1', 'SumaPagos'].item()
C2 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C2', 'SumaPagos'].item()
C3 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C3', 'SumaPagos'].item()
C4 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C4', 'SumaPagos'].item()
B = pagos_totales_brad.loc[pagos_totales_brad['Division']=='B', 'SumaPagos'].item()

In [ ]:
def func(row):
    if row['TIPO_ASIGANCION'] == 'C0 4PV':
        return np.where((C04PV>0) & (C04PV<=c04pv1),arancel_4pv1,
                        np.where((C04PV>c04pv1) & (C04PV<=c04pv2),arancel_4pv2,
                                 np.where((C04PV>c04pv2) & (C04PV<=c04pv3),arancel_4pv3,arancel_4pv3+.03)))
    elif row['TIPO_ASIGANCION'] == 'C0 5PV':
        return np.where((C05PV>0) & (C05PV<=c05pv1),arancel_5pv1,
                        np.where((C05PV>c05pv1) & (C05PV<=c05pv2),arancel_5pv2,
                                 np.where((C05PV>c05pv2) & (C05PV<=c05pv3),arancel_5pv3,arancel_5pv3+.03)))
    elif row['TIPO_ASIGANCION'] == 'C0 6PV':
        return np.where((C06PV>0) & (C06PV<=c06pv1),arancel_6pv1,
                        np.where((C06PV>c06pv1) & (C06PV<=c06pv2),arancel_6pv2,
                                 np.where((C06PV>c06pv2) & (C06PV<=c06pv3),arancel_6pv3,arancel_6pv3+.03)))
    elif row['TIPO_ASIGANCION'] == 'A':
        return np.where((A>0) & (A<=a1),arancel_a1,
                        np.where((A>a1) & (A<=a2),arancel_a2,
                                 np.where((A>a2) & (A<=a3),arancel_a3,arancel_a3+.03)))
    elif row['TIPO_ASIGANCION'] == 'B':
        return np.where((B>0) & (B<=b1),arancel_b1,
                        np.where((B>b1) & (B<=b2),arancel_b2,
                                 np.where((B>b2) & (B<=b3),arancel_b3,arancel_b3+.03)))
    elif row['TIPO_ASIGANCION'] == 'C1':
        return np.where((C1>0) & (C1<=c11),arancel_c11,
                        np.where((C1>c11) & (C1<=c12),arancel_c12,
                                 np.where((C1>c12) & (C1<=c13),arancel_c13,arancel_c13+.03)))
    elif row['TIPO_ASIGANCION'] == 'C2':
        return np.where((C2>0) & (C2<=c21),arancel_c21,
                        np.where((C2>c21) & (C2<=c22),arancel_c22,
                                 np.where((C2>c22) & (C2<=c23),arancel_c23,arancel_c23+.03)))
    elif row['TIPO_ASIGANCION'] == 'C3':
        return np.where((C3>0) & (C3<=c31),arancel_c31,
                        np.where((C3>c31) & (C3<=c32),arancel_c32,
                                 np.where((C3>c32) & (C3<=c33),arancel_c33,arancel_c33+.03)))
    elif row['TIPO_ASIGANCION'] == 'C4':
        return np.where((C4>0) & (C4<=c41),arancel_c41,
                        np.where((C4>c41) & (C4<=c42),arancel_c42,
                                 np.where((C4>c42) & (C4<=c43),arancel_c43,arancel_c43+.03)))
    
    else:
        return 0

pagos_brad2['Arancel'] = pagos_brad2.apply(func, axis=1)

In [ ]:
pagos_brad2['Facturacion'] = [x*y for x,y in zip(pagos_brad2['MONTO'],pagos_brad2['Arancel'])]

In [ ]:
#Hacemos la union consolidado y pagos
consoliado_pagos_brad = pd.merge(consolidado_brad_mes,pagos_brad2,how='left',left_on=['credito'],right_on=['NUMERO_TARJETA'])
try: consoliado_pagos_brad['fecha_promesa'] = pd.to_datetime(consoliado_pagos_brad['fecha_promesa'],dayfirst=False)
except: consoliado_pagos_brad['fecha_promesa'] = consoliado_pagos_brad['fecha_gestion']

In [ ]:
#Condicionamos a los gestores cuyas promesas y pagos estan en un plazo de 3 dias
consoliado_pagos_brad['PagoUnico'] = [np.where((z < x+pd.Timedelta(4, unit='D')) & ((z > x-pd.Timedelta(3, unit='D') )), y, 0) for x,y,z in zip(consoliado_pagos_brad['fecha_promesa'],consoliado_pagos_brad['MONTO'],consoliado_pagos_brad['FECHA_PAGO'])]
consoliado_pagos_brad['PagoUnico'] = [x.item() for x in consoliado_pagos_brad['PagoUnico']]

In [ ]:
consoliado_pagos_brad['PagoReal'] = [x*y for x,y in zip(consoliado_pagos_brad['PagoUnico'],consoliado_pagos_brad['Arancel'])]
consoliado_pagos_brad2 = consoliado_pagos_brad.drop_duplicates(subset=['credito','MONTO','FECHA_PAGO','usuario'])

In [ ]:
#Obtenemos datos de las promesas de Bradesco
prom_brad_div_mes = pd.pivot_table(consoliado_pagos_brad2,index='division',values=['monto_promesa'], aggfunc = ['count',np.sum], margins=True,dropna=False)
prom_brad_div_mes = pd.DataFrame(prom_brad_div_mes.to_records())
prom_brad_div_mes.columns = ['Division','NumeroPromesas','SumaPromesas']
prom_brad_div_mes.fillna(0,inplace=True)

In [ ]:
#Obtenemos las promesas restantes a partir del día de hoy
restantes_brad = consoliado_pagos_brad2.loc[consoliado_pagos_brad2['fecha_promesa']>=ayer]
promesas_restantes_brad = pd.pivot_table(restantes_brad,index=['division'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
promesas_restantes_brad = pd.DataFrame(promesas_restantes_brad.to_records())
promesas_restantes_brad.columns = ['Division','PromesasRestantes']
promesas_restantes_brad.fillna(0,inplace=True)
promesas_restantes_brad

In [ ]:
pagos_totales_brad2 = pd.pivot_table(pagos_brad2,index=['TIPO_ASIGANCION'],values=['MONTO','Facturacion'],aggfunc=['count',np.sum],dropna=False)
pagos_totales_brad2 = pd.DataFrame(pagos_totales_brad2.to_records())
pagos_totales_brad2 = pagos_totales_brad2.iloc[:,[0,1,4,3]]
pagos_totales_brad2.columns = ['Division','NumeroPagos','SumaPagos','SumaFacturacion']
pagos_totales_brad2.fillna(0,inplace=True)
pagos_totales_brad2['FechaFact'] = max(pagos_brad2['FECHA_PAGO'])

In [ ]:
final_brad = pd.merge(pagos_totales_brad2,prom_brad_div_mes,how='left',on=['Division'])
final_brad = pd.merge(final_brad,promesas_restantes_brad,how='left',on=['Division'])
final_brad.fillna(0,inplace=True)
final_brad.insert(loc=0,column='Cliente',value='Bradesco')
final_brad

# Reporte Banco Azteca  

In [ ]:
#Hacemos la consulta referente a las gestiones de Banco Azteca
sql_gest_baz = cursor.callproc('baz_rpt_gestiones_detallado', [mes,today])
for result in cursor.stored_results():
     gestion_baz = pd.DataFrame(result.fetchall())
gestion_baz.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_baz = gestion_baz.loc[gestion_baz['dictamen']=='PROMESA']
gestion_baz['monto_promesa'] = pd.to_numeric(gestion_baz['monto_promesa'])
gestion_baz['fecha_promesa'] = pd.to_datetime(gestion_baz['fecha_promesa'],dayfirst=False)
#gestion_baz.head(3)

In [ ]:
#Obtenemos datos de las promesas de Banco Azteca
gestion_baz = gestion_baz.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
prom_baz_div_mes = pd.pivot_table(gestion_baz,index=['dictamen'],values=['unegocio_id','monto_promesa'],aggfunc=('count',np.sum))
prom_baz_div_mes = pd.DataFrame(prom_baz_div_mes.to_records())
prom_baz_div_mes = prom_baz_div_mes.iloc[:,[0,1,2]]
prom_baz_div_mes.columns = ['Division','NumeroPromesas','SumaPromesas']
prom_baz_div_mes.fillna(0,inplace=True)
prom_baz_div_mes = prom_baz_div_mes.replace('PROMESA','COBRANZA')
prom_baz_div_mes

In [ ]:
#Obtenemos las promesas restantes a partir del día de hoy
restantes_baz = gestion_baz.loc[gestion_baz['fecha_promesa']>=ayer]

promesas_restantes_baz = pd.pivot_table(restantes_baz,index=['dictamen'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
promesas_restantes_baz = pd.DataFrame(promesas_restantes_baz.to_records())
promesas_restantes_baz.columns = ['Division','PromesasRestantes']
promesas_restantes_baz = promesas_restantes_baz.replace('PROMESA','COBRANZA')
promesas_restantes_baz.fillna(0,inplace=True)
promesas_restantes_baz

In [ ]:
pagos_baz2['division'] = 'COBRANZA'
pagos_baz2['FNFACTURACION'] = [x*.2 for x in pagos_baz2['FNMONTOPAGADO']]
pagos_totales_baz = pd.pivot_table(pagos_baz2,index=['division'],values=['FNMONTOPAGADO'],aggfunc=['count',np.sum],dropna=False)
pagos_totales_baz = pd.DataFrame(pagos_totales_baz.to_records())
#pagos_totales_baz = pagos_totales_baz.iloc[:,[0,1,4,3]]
pagos_totales_baz.columns = ['Division','NumeroPagos','SumaPagos']
#pagos_totales_baz.fillna(0,inplace=True)
#pagos_totales_baz['FechaFact'] = ayer
pagos_totales_baz

In [ ]:
fact_baz['Total Recuperado'] = np.where(fact_baz['Inicio de Semana']<mes3,fact_baz['Total Recuperado']*.85,fact_baz['Total Recuperado'])
fact_baz['Total Comisión'] = np.where(fact_baz['Inicio de Semana']<mes3,fact_baz['Total Comisión']*.85,fact_baz['Total Comisión'])

In [ ]:
#Hacemos la consulta de pagos y facturacion
fact_baz['Division2'] = 'COBRANZA'
pagos_fact_baz = pd.pivot_table(fact_baz,index=['Division2'],values=['Total Recuperado','Total Comisión'],aggfunc=[np.sum])
pagos_fact_baz = to_dataframe(pagos_fact_baz)
pagos_fact_baz = pagos_fact_baz.iloc[:,[0,2,1]]
pagos_fact_baz.columns = ['Division','SumaPagos','SumaFacturacion']
pagos_fact_baz['FechaFact'] = max(fact_baz['Final de Semana'])
pagos_fact_baz

In [ ]:
final_baz = pd.merge(pagos_fact_baz,prom_baz_div_mes,how='left',on=['Division'])
final_baz = pd.merge(final_baz,promesas_restantes_baz,how='left',on=['Division'])
final_baz.fillna(0,inplace=True)
final_baz.insert(loc=0,column='Cliente',value='Banco Azteca')
final_baz

# Reporte Credifiel

In [ ]:
#Hacemos la consulta referente a las gestiones de Credifiel
sql_gest_cred = cursor.callproc('credifiel_rpt_gestiones_detallado', [mes,today])
for result in cursor.stored_results():
    gestion_cred = pd.DataFrame(result.fetchall())
gestion_cred.columns = ['folio_gestion','unegocio_id','fecha_gestion','hora_gestion','credito','nombre_credito','telefono','usuario','nombre_usuario','accion','resultado','fecha_promesa','monto_promesa','comentarios','accion_credifiel','resultado_credifiel','dictamen']
gestion_cred['monto_promesa'] = pd.to_numeric(gestion_cred['monto_promesa'])
gestion_cred = gestion_cred.loc[gestion_cred['accion_credifiel']=='PDP']
gestion_cred['credito'] = [x[3:] for x in gestion_cred['credito']]
gestion_cred['credito'] = pd.to_numeric(gestion_cred['credito'])
#gestion_cred.head(3)

In [ ]:
#Hacemos la consulta de la asignacion
sheet_asig_cred = os.listdir('/home/estadistico/Documents/Erick/Credifiel/Asignacion/')
asignacion_cred = pd.DataFrame()
for i in sheet_asig_cred:
    asignacion_cred_new = pd.read_excel('/home/estadistico/Documents/Erick/Credifiel/Asignacion/'+i+'')
    asignacion_cred = pd.concat([asignacion_cred,asignacion_cred_new],ignore_index=True)
asignacion_cred = asignacion_cred.drop_duplicates(subset=['CREDITO','SALDO TOTAL REAL'])
asignacion_cred['CREDITO'] = pd.to_numeric(asignacion_cred['CREDITO'])

In [ ]:
#Hacemos el consolidado de la asignacion y promesado
consolidado_cred = pd.merge(gestion_cred,asignacion_cred,how='left',left_on=['credito'],right_on=['CREDITO'])
cols = ['monto_promesa','unegocio_id']
consolidado_cred[cols] = consolidado_cred[cols].apply(pd.to_numeric, errors='coerce')
consolidado_cred = consolidado_cred.loc[consolidado_cred['monto_promesa']>50]
consolidado_cred = consolidado_cred[(consolidado_cred['monto_promesa']<500000)]
consolidado_cred['fecha_promesa'] = pd.to_datetime(consolidado_cred['fecha_promesa'],dayfirst=False)
consolidado_cred.replace('CONTACTO','EXTRA',inplace=True)

In [ ]:
pagos_cred3 = pd.merge(pagos_cred2,consolidado_cred[['credito','nombre_usuario','fecha_promesa','monto_promesa','ARANCEL']],how='left',left_on=['ID'],right_on=['credito'])
pagos_cred3['ARANCEL'].fillna(np.mean(pagos_cred3['ARANCEL']),inplace=True)
pagos_cred3['PagoFacturacion'] = [x*y for x,y in zip(pagos_cred3['ARANCEL'],pagos_cred3['IMPORTE'])]
pagos_cred3['division'] = 'EXTRA'
pagos_cred3 = pagos_cred3.drop_duplicates(subset=['ID','IMPORTE'])

In [ ]:
#Obtenemos datos de las promesas de Bradesco
prom_cred_div_mes = pd.pivot_table(consolidado_cred,index=['dictamen'],values=['monto_promesa'], aggfunc=['count',np.sum], dropna=False)
prom_cred_div_mes = pd.DataFrame(prom_cred_div_mes.to_records())
#prom_cred_div_mes = prom_cred_div_mes.iloc[:,[0,4,8,6,5]]
prom_cred_div_mes.columns = ['Division','NumeroPromesas','SumaPromesas']
#prom_cred_div_mes = prom_cred_div_mes.sort_values(by=['SumaPago'],ascending =False)
#prom_cred_div_mes = prom_cred_div_mes.reset_index(drop=True)
#prom_cred_div_mes.fillna(0,inplace=True)
#prom_brad_div_mes[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = prom_brad_gestor_mes[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
prom_cred_div_mes

In [ ]:
#Obtenemos las promesas restantes a partir del día de hoy
restantes_cred = consolidado_cred.loc[consolidado_cred['fecha_promesa']>=ayer]

promesas_restantes_cred = pd.pivot_table(restantes_cred,index=['dictamen'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
promesas_restantes_cred = pd.DataFrame(promesas_restantes_cred.to_records())
#promesas_restantes_brad = promesas_restantes_brad.iloc[:,[0,3,8,6,5]]
promesas_restantes_cred.columns = ['Division','PromesasRestantes']
promesas_restantes_cred.fillna(0,inplace=True)

#posible cambio
#promesas_restantes_brad = promesas_restantes_brad.sort_values(by='SumaPagos',ascending=False)
#promesas_restantes_brad[['SumaPromesas']] = promesas_restantes_brad[['SumaPromesas']].applymap("{0:.2f}".format)
promesas_restantes_cred

In [ ]:
pagos_totales_cred = pd.pivot_table(pagos_cred3,index=['division'],values=['IMPORTE','PagoFacturacion'],aggfunc=['count',np.sum],dropna=False)
pagos_totales_cred = pd.DataFrame(pagos_totales_cred.to_records())
pagos_totales_cred = pagos_totales_cred.iloc[:,[0,1,3,4]]
pagos_totales_cred.columns = ['Division','NumeroPagos','SumaPagos','SumaFacturacion']
pagos_totales_cred.fillna(0,inplace=True)
#pagos_totales_cred[['SumaPagos','SumaFacturacion']] = pagos_totales_cred[['SumaPagos','SumaFacturacion']].applymap("${0:,.2f}".format)
pagos_totales_cred['FechaFact'] = max(pagos_cred3['FECHA'])
pagos_totales_cred

In [ ]:
final_cred = pd.merge(pagos_totales_cred,prom_cred_div_mes,how='left',on=['Division'])
final_cred = pd.merge(final_cred,promesas_restantes_cred,how='left',on=['Division'])
final_cred.fillna(0,inplace=True)
final_cred.insert(loc=0,column='Cliente',value='Credifiel')
final_cred = final_cred.replace('EXTRA','COBRANZA')
final_cred

# Reporte final Naucalpan

In [ ]:
final_naucalpan = pd.concat([final_liv,final_brad],ignore_index=True)
final_naucalpan = pd.concat([final_naucalpan,final_cred],ignore_index=True)
final_naucalpan = pd.concat([final_naucalpan,final_baz],ignore_index=True)
final_naucalpan.insert(loc=0,column='Sucursal',value='Naucalpan')
final_naucalpan.insert(loc=0,column='Pais',value='Mexico')
final_naucalpan['TendenciaCierre1'] = [(30/x.day)*y for x,y in zip(final_naucalpan['FechaFact'],final_naucalpan['SumaFacturacion'])]
final_naucalpan = final_naucalpan.replace('EXTRA','Cobranza')
final_naucalpan = final_naucalpan.replace('COBRANZA','Cobranza')
final_naucalpan.fillna('',inplace=True)
final_naucalpan

In [ ]:
final_naucalpan.to_csv(''+ruta_guardar+'/Reporte Sucursal/Reporte sucursal '+today2+'.csv',index=False,header=True)